<h1>Arquivo de Refactoring do Modelo</h1>

In [1]:
#from pandas import DataFrame, Series
import pandas as pd

import numpy as np

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# Importa o código principal
import sys
sys.path.append('C:\\Users\\Danilo Santos\\Desktop\\Qualificação PPGCC\\abordagem\\RFNS')

# from grimoire.EnginneringForest import EnginneringForest
from grimoire.ClassifierEnginneringForest import ClassifierEnginneringForest
from grimoire.LoggerEnginnering import LoggerEnginnering

In [3]:
df_heart = pd.read_csv('../datasets/heart.csv',
                       engine='c', 
                       memory_map=True, 
                       low_memory=True)

X=df_heart[['age', 'sex', 'cp', 'trestbps',  'chol', 'fbs', 'restecg', 
            'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
# Labels
y=df_heart['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                        random_state=100, 
                                                        shuffle=True, 
                                                        stratify=y)

In [4]:
X_train.shape

(212, 13)

In [10]:
print(list(X_train.index))

[241, 246, 97, 60, 298, 217, 42, 88, 7, 143, 126, 52, 187, 152, 64, 68, 194, 164, 282, 247, 18, 165, 111, 239, 156, 193, 148, 46, 45, 142, 99, 202, 40, 183, 184, 210, 284, 188, 205, 27, 288, 297, 200, 124, 37, 106, 78, 41, 132, 31, 80, 243, 190, 294, 245, 206, 220, 154, 289, 162, 105, 237, 275, 110, 213, 19, 95, 3, 287, 59, 4, 84, 71, 238, 234, 171, 159, 32, 293, 253, 161, 235, 75, 145, 69, 54, 240, 266, 141, 15, 216, 79, 29, 236, 219, 185, 35, 221, 296, 177, 85, 197, 109, 12, 6, 153, 103, 291, 8, 233, 98, 255, 121, 56, 91, 283, 286, 76, 226, 186, 62, 222, 58, 175, 107, 10, 176, 150, 207, 118, 279, 120, 250, 229, 163, 271, 264, 270, 50, 211, 260, 280, 26, 16, 23, 157, 276, 230, 299, 261, 160, 182, 100, 292, 149, 96, 198, 20, 131, 204, 65, 208, 290, 102, 70, 195, 262, 209, 43, 215, 138, 125, 47, 257, 191, 137, 123, 104, 39, 170, 242, 254, 61, 203, 168, 155, 134, 228, 227, 267, 17, 13, 86, 89, 249, 281, 21, 128, 139, 136, 36, 302, 112, 49, 5, 1, 72, 201, 151, 269, 9, 83]


<h1>Procura os erros de indices nas etapas de treinamento</h1>

In [22]:
class EnginneringForest(ClassifierEnginneringForest):
    
    __slots__ = ('estimators_', 'select_features_', 'group_features_', 
                 'df_predict_', 'n_features_', 'n_samples_', 'name_features_',
                 'prefix_column_predict', 'logger', 'blockpredict', 'autoclean')
    
    def __init__(self, select_features: int, reset_log=False, name_log='enginnering.log'):
        if type(select_features) != int:
            raise TypeError('Expectd value int in select_features')
        
        self.estimators_ = []
        self.select_features_ = select_features
        self.group_features_ = []
        self.df_predict_ = []
        self.n_features_ = 0
        self.n_samples_ = 0
        self.name_features_ = []
        self.logger = LoggerEnginnering(name='enginnering', 
                                        log_file=name_log,
                                        drop_old=reset_log)
        self.blockpredict = []
        self.autoclean = False
        super().__init__()
        
    def __del__(self):
        del self.estimators_
        del self.select_features_
        del self.group_features_
        del self.df_predict_
        del self.n_features_
        del self.n_samples_
        del self.name_features_
        #del self.prefix_column_predict

    def build(self, features_set: list) -> None:
        """ Cria um vetor com o número de árvores igual ao número de 
            subconjuntos possíveis """
        self.group_features_ = self.get_arrangement_features(features_set,
                                                             self.select_features_)
        self.group_features_ = self.get_pack_nparray(self.group_features_)
        n_estimator = len(self.group_features_)
        self.estimators_ = self.make_lote_base_estimator(n_estimator)
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
        #print('N group features: ', len(self.group_features_))
        #print('N estimators: ', len(self.estimators_))
        #print('Group features: \n{0}'.format(self.group_features_))

    def train(self, group_feature: list, estimator):
        msg = 'Training subset = {0}, Timing = {1}, Size (Kb) = {2}'
        start_train = time.time()
        
        subset_xdata, subset_ydata = self.get_subset(group_feature)
        fit_ = estimator.fit(subset_xdata, subset_ydata)
        
        #print('Train = {0}, Length X = {1}, Length Y = {2}'.format(group_feature, subset_xdata.shape, subset_ydata.shape))
        
        #del subset_xdata
        #del subset_ydata
        
        end_train = time.time()
        self.logger.add('debug',msg.format(group_feature, 
                                           (end_train - start_train),
                                           self.get_size_estimator(fit_)))
        
        return fit_

    def fit(self, X, y) -> None:
        from pandas import DataFrame, Series
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
        if not isinstance(y, Series):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
            
        self.n_samples_, self.n_features_ = X.shape
        self.name_features_ = X.columns
        
        self.train_X = X
        self.train_y = y
        
        #print('N samples: ',  self.n_samples_)
        #print('N features: ', self.n_features_)
        #print('Name features: ', self.name_features_)
        
        # Cria a floresta
        self.build(features_set=self.name_features_)

        # Treina as arvores individualmente
        self.estimators_ = [self.train(subset_feature, estimator) 
                            for subset_feature, 
                                estimator in zip(self.group_features_, 
                                                 self.estimators_)]
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
        if self.autoclean:
            del self.train_X
            del self.train_y

In [23]:
model_eg = EnginneringForest(3)

In [24]:
model_eg.fit(X_train, y_train)

In [28]:
subset_x, subset_y = model_eg.get_subset(['sex','restecg','thalach'])

In [29]:
subset_x.head()

,sex,restecg,thalach
241,0,1,143
246,0,0,150
97,1,1,147
60,0,0,130
298,0,1,123


In [30]:
subset_y.head()

241    0
246    0
97     1
60     1
298    0
Name: target, dtype: int64

In [31]:
print(subset_x.shape, subset_y.shape)

(212, 3) (212,)


In [35]:
for x, y in zip(subset_x.index, subset_y.index):
                print('{0} == {1}'.format(x,y))

241 == 241
246 == 246
97 == 97
60 == 60
298 == 298
217 == 217
42 == 42
88 == 88
7 == 7
143 == 143
126 == 126
52 == 52
187 == 187
152 == 152
64 == 64
68 == 68
194 == 194
164 == 164
282 == 282
247 == 247
18 == 18
165 == 165
111 == 111
239 == 239
156 == 156
193 == 193
148 == 148
46 == 46
45 == 45
142 == 142
99 == 99
202 == 202
40 == 40
183 == 183
184 == 184
210 == 210
284 == 284
188 == 188
205 == 205
27 == 27
288 == 288
297 == 297
200 == 200
124 == 124
37 == 37
106 == 106
78 == 78
41 == 41
132 == 132
31 == 31
80 == 80
243 == 243
190 == 190
294 == 294
245 == 245
206 == 206
220 == 220
154 == 154
289 == 289
162 == 162
105 == 105
237 == 237
275 == 275
110 == 110
213 == 213
19 == 19
95 == 95
3 == 3
287 == 287
59 == 59
4 == 4
84 == 84
71 == 71
238 == 238
234 == 234
171 == 171
159 == 159
32 == 32
293 == 293
253 == 253
161 == 161
235 == 235
75 == 75
145 == 145
69 == 69
54 == 54
240 == 240
266 == 266
141 == 141
15 == 15
216 == 216
79 == 79
29 == 29
236 == 236
219 == 219
185 == 185
35 == 35
221 == 

<h1>Procura os erros de indices na etapa de predição</h1>

In [4]:
class EnginneringForest(ClassifierEnginneringForest):
    
    __slots__ = ('estimators_', 'select_features_', 'group_features_', 
                 'df_predict_', 'n_features_', 'n_samples_', 'name_features_',
                 'prefix_column_predict', 'logger', 
                 'blockpredict', 'autoclean')
    
    def __init__(self, select_features: int, reset_log=False, name_log='enginnering.log'):
        if type(select_features) != int:
            raise TypeError('Expectd value int in select_features')
        
        self.estimators_ = []
        self.select_features_ = select_features
        self.group_features_ = []
        self.df_predict_ = []
        self.n_features_ = 0
        self.n_samples_ = 0
        self.name_features_ = []
        self.logger = LoggerEnginnering(name='enginnering', 
                                        log_file=name_log,
                                        drop_old=reset_log)
        self.blockpredict = []
        self.autoclean = False
        super().__init__()
        
    def __del__(self):
        del self.estimators_
        del self.select_features_
        del self.group_features_
        del self.df_predict_
        del self.n_features_
        del self.n_samples_
        del self.name_features_

    def build(self, features_set: list) -> None:
        """ Cria um vetor com o número de árvores igual ao número de 
            subconjuntos possíveis """
        self.group_features_ = self.get_arrangement_features(features_set,
                                                             self.select_features_)
        self.group_features_ = self.get_pack_nparray(self.group_features_)
        n_estimator = len(self.group_features_)
        self.estimators_ = self.make_lote_base_estimator(n_estimator)
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
    def train(self, group_feature: list, estimator):
        msg = 'Training subset = {0}, Timing = {1}, Size (Kb) = {2}'
        start_train = time.time()
        
        subset_xdata, subset_ydata = self.get_subset(group_feature)
        fit_ = estimator.fit(subset_xdata, subset_ydata)
        
        end_train = time.time()
        self.logger.add('debug',msg.format(group_feature, 
                                           (end_train - start_train),
                                           self.get_size_estimator(fit_)))
        
        return fit_

    def fit(self, X, y) -> None:
        from pandas import DataFrame, Series
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
        if not isinstance(y, Series):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
            
        self.n_samples_, self.n_features_ = X.shape
        self.name_features_ = X.columns
        self.train_X = X
        self.train_y = y
        
        self.build(features_set=self.name_features_)

        self.estimators_ = [self.train(subset_feature, estimator) 
                            for subset_feature, 
                                estimator in zip(self.group_features_, 
                                                 self.estimators_)]
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
        if self.autoclean:
            del self.train_X
            del self.train_y
    
    def predict(self, X) -> list:
        from pandas import DataFrame
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
       
        self.predict_X = X

        self.logger.add('debug',
                        'Size predict = {0}, N estimators = {1}'.format(self.predict_X.shape, 
                                                                        len(self.estimators_)))                                  
        
        for x_, y_ in self.get_block_fit():
            self.logger.add('debug','Block Limit = ({}, {})'.format(x_, y_))
            
            dfsub = self.predict_X.iloc[x_:y_+1]
            block_predict = []
            
            print('Size DfSub = {0}'.format(dfsub.shape))
            
            for subset_feature, estimator in zip(self.group_features_, self.estimators_):
                self.logger.add('debug', 'Subset predict = {0}'.format(subset_feature))
                subset_test = dfsub.loc[:, subset_feature]
                block_predict.append(estimator.predict(subset_test))
            
            block_predict = np.matrix(block_predict)
            self.logger.add('debug', "Shape One = {0}".format(block_predict.shape))
            
            block_predict = block_predict.T
            self.logger.add('debug', "Shape Two = {0}".format(block_predict.shape))
            self.logger.add('debug', "Block predict \n{0}".format(block_predict))
            
            # Guarda a matrix de predição
            #np.savetxt('dump_sf_{0}_bp_{1}'.format(self.select_features_, (x_,y_)), 
            #           block_predict, delimiter=",")
            
            self.blockpredict = pd.DataFrame(block_predict)
            
            # chama o voting na matriz de predições
            block_voting = self.voting(block_predict)
            self.logger.add('debug', "Block voting data \n{0}".format(str(block_voting)))
            self.logger.add('debug', "Block voting len {0}".format(len(block_voting)))
            self.df_predict_.extend(block_voting)
        
        return self.df_predict_
    
    def voting(self, data) -> list:
        final_predict = []
        for instance in data:
            marjotiry = (instance.sum() >= (instance.shape[1] / 2)) and 1 or 0
            final_predict.append(marjotiry)
        return final_predict

In [5]:
model = EnginneringForest(3, name_log='test2.txt')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Size DfSub = (91, 13)


In [6]:
model.blockpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,276,277,278,279,280,281,282,283,284,285
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0


In [7]:
print(list(y_pred))

[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0]


In [8]:
print(list(y_test))

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]


In [11]:
print(list(model.blockpredict.iloc[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]


In [12]:
len(y_pred)

91

In [13]:
list(model.blockpredict.iloc[0]).count(1)

63

In [14]:
list(model.blockpredict.iloc[0]).count(0)

223

In [15]:
(model.blockpredict.iloc[0].sum() >= (model.blockpredict.iloc[0].shape[1] / 2)) and 1 or 0

IndexError: tuple index out of range

In [19]:
mac = accuracy_score(y_test, y_pred)
print("Accuracy:", mac)

Accuracy: 0.8571428571428571


In [24]:
model.blockpredict.iloc[6].shape

(286,)

In [49]:
model_eg = EnginneringForest(3)

In [50]:
model_eg.fit(X_train, y_train)

In [51]:
y_pred = model_eg.predict(X_test)

Size DfSub = (90, 13)


In [52]:
model_eg.blockpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,276,277,278,279,280,281,282,283,284,285
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,1,1,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0


In [53]:
model_eg.blockpredict.shape

(90, 286)

In [54]:
block = model_eg.blockpredict

In [55]:
block.insert(loc=286, column='predict', value=y_pred)

In [56]:
block.insert(loc=287, column='real', value=y_test)

In [57]:
block.head()

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,predict,real
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1.0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,NaN
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1,0,1,1,1,1,0,1,1,0,...,0,0,1,0,0,1,0,0,1,NaN


In [70]:
block = X_test

In [71]:
block.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,real,thal
214,56,1,0,125,249,1,0,144,1,1.2,1,1,0,2
25,71,0,1,160,302,0,1,162,0,0.4,2,2,1,2
22,42,1,0,140,226,0,1,178,0,0.0,2,0,1,2
256,58,1,0,128,259,0,0,130,1,3.0,1,2,0,3
93,54,0,1,132,288,1,0,159,1,0.0,2,1,1,2


In [62]:
block.shape

(91, 13)

In [63]:
len(y_test)

91

In [64]:
type(y_test)

pandas.core.series.Series

In [73]:
print(list(y_pred.index))

TypeError: 'builtin_function_or_method' object is not iterable

In [66]:
block.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,real,thal
214,56,1,0,125,249,1,0,144,1,1.2,1,1,0,2
25,71,0,1,160,302,0,1,162,0,0.4,2,2,1,2
22,42,1,0,140,226,0,1,178,0,0.0,2,0,1,2
256,58,1,0,128,259,0,0,130,1,3.0,1,2,0,3
93,54,0,1,132,288,1,0,159,1,0.0,2,1,1,2


In [36]:
print(list(X_test.index))

[214, 25, 22, 256, 93, 114, 258, 122, 28, 232, 33, 199, 77, 146, 133, 273, 48, 127, 38, 223, 2, 166, 172, 34, 180, 92, 113, 278, 251, 101, 244, 231, 140, 81, 144, 265, 225, 67, 115, 119, 218, 44, 30, 173, 169, 224, 259, 66, 212, 73, 285, 179, 248, 51, 129, 74, 116, 272, 135, 130, 94, 82, 87, 158, 57, 117, 90, 55, 301, 167, 181, 14, 174, 192, 274, 11, 53, 0, 263, 300, 189, 252, 268, 108, 277, 147, 63, 24, 295, 196, 178]


In [37]:
print(list(y_test.index))

[214, 25, 22, 256, 93, 114, 258, 122, 28, 232, 33, 199, 77, 146, 133, 273, 48, 127, 38, 223, 2, 166, 172, 34, 180, 92, 113, 278, 251, 101, 244, 231, 140, 81, 144, 265, 225, 67, 115, 119, 218, 44, 30, 173, 169, 224, 259, 66, 212, 73, 285, 179, 248, 51, 129, 74, 116, 272, 135, 130, 94, 82, 87, 158, 57, 117, 90, 55, 301, 167, 181, 14, 174, 192, 274, 11, 53, 0, 263, 300, 189, 252, 268, 108, 277, 147, 63, 24, 295, 196, 178]


In [38]:
len(X_test.index)

91

In [43]:
X_test.iloc[10:20]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
33,54,1,2,125,273,0,0,152,0,0.5,0,1,2
199,65,1,0,110,248,0,0,158,0,0.6,2,2,1
77,59,1,1,140,221,0,1,164,1,0.0,2,0,2
146,44,0,2,118,242,0,1,149,0,0.3,1,1,2
133,41,1,1,110,235,0,1,153,0,0.0,2,0,2
273,58,1,0,100,234,0,1,156,0,0.1,2,1,3
48,53,0,2,128,216,0,0,115,0,0.0,2,0,0
127,67,0,2,152,277,0,1,172,0,0.0,2,1,2
38,65,0,2,155,269,0,1,148,0,0.8,2,0,2
223,56,0,0,200,288,1,0,133,1,4.0,0,2,3


<h1>Procura os erros no voting</h1>

In [61]:
class EnginneringForest(ClassifierEnginneringForest):
    
    __slots__ = ('estimators_', 'select_features_', 'group_features_', 
                 'df_predict_', 'n_features_', 'n_samples_', 'name_features_',
                 'prefix_column_predict', 'logger', 
                 'blockpredict', 'autoclean', 'matrix_predict')
    
    def __init__(self, select_features: int, reset_log=False, name_log='enginnering.log'):
        if type(select_features) != int:
            raise TypeError('Expectd value int in select_features')
        
        self.estimators_ = []
        self.select_features_ = select_features
        self.group_features_ = []
        self.df_predict_ = []
        self.n_features_ = 0
        self.n_samples_ = 0
        self.name_features_ = []
        self.logger = LoggerEnginnering(name='enginnering', 
                                        log_file=name_log,
                                        drop_old=reset_log)
        self.blockpredict = []
        self.matrix_predict = [[]]
        self.autoclean = False
        super().__init__()
        
    def __del__(self):
        del self.estimators_
        del self.select_features_
        del self.group_features_
        del self.df_predict_
        del self.n_features_
        del self.n_samples_
        del self.name_features_

    def build(self, features_set: list) -> None:
        """ Cria um vetor com o número de árvores igual ao número de 
            subconjuntos possíveis """
        self.group_features_ = self.get_arrangement_features(features_set,
                                                             self.select_features_)
        self.group_features_ = self.get_pack_nparray(self.group_features_)
        n_estimator = len(self.group_features_)
        self.estimators_ = self.make_lote_base_estimator(n_estimator)
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
    def train(self, group_feature: list, estimator):
        msg = 'Training subset = {0}, Timing = {1}, Size (Kb) = {2}'
        start_train = time.time()
        
        subset_xdata, subset_ydata = self.get_subset(group_feature)
        fit_ = estimator.fit(subset_xdata, subset_ydata)
        
        end_train = time.time()
        self.logger.add('debug',msg.format(group_feature, 
                                           (end_train - start_train),
                                           self.get_size_estimator(fit_)))
        
        return fit_

    def fit(self, X, y) -> None:
        from pandas import DataFrame, Series
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
        if not isinstance(y, Series):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
            
        self.n_samples_, self.n_features_ = X.shape
        self.name_features_ = X.columns
        self.train_X = X
        self.train_y = y
        
        self.build(features_set=self.name_features_)

        self.estimators_ = [self.train(subset_feature, estimator) 
                            for subset_feature, 
                                estimator in zip(self.group_features_, 
                                                 self.estimators_)]
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
        if self.autoclean:
            del self.train_X
            del self.train_y
    
    def predict(self, X) -> list:
        from pandas import DataFrame
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
       
        self.predict_X = X

        self.logger.add('debug',
                        'Size predict = {0}, N estimators = {1}'.format(self.predict_X.shape, 
                                                                        len(self.estimators_)))                                  
        
        for x_, y_ in self.get_block_fit():
            self.logger.add('debug','Block Limit = ({}, {})'.format(x_, y_))
            
            dfsub = self.predict_X.iloc[x_:y_+1]
            block_predict = []
            
            print('Size DfSub = {0}'.format(dfsub.shape))
            
            for subset_feature, estimator in zip(self.group_features_, self.estimators_):
                self.logger.add('debug', 'Subset predict = {0}'.format(subset_feature))
                subset_test = dfsub.loc[:, subset_feature]
                block_predict.append(estimator.predict(subset_test))
            
            block_predict = np.matrix(block_predict)
            self.matrix_predict = block_predict
            self.logger.add('debug', "Shape One = {0}".format(block_predict.shape))
            
            block_predict = block_predict.T
            self.logger.add('debug', "Shape Two = {0}".format(block_predict.shape))
            self.logger.add('debug', "Block predict \n{0}".format(block_predict))
            
            # Guarda a matrix de predição
            #np.savetxt('dump_sf_{0}_bp_{1}'.format(self.select_features_, (x_,y_)), 
            #           block_predict, delimiter=",")
            
            self.blockpredict = pd.DataFrame(block_predict)
            
            # chama o voting na matriz de predições
            block_voting = self.voting(block_predict)
            self.logger.add('debug', "Block voting data \n{0}".format(str(block_voting)))
            self.logger.add('debug', "Block voting len {0}".format(len(block_voting)))
            self.df_predict_.extend(block_voting)
        
        return self.df_predict_
    
    def voting(self, data) -> list:
        final_predict = []
        i = 0
        for instance in data:
            cz = instance.tolist()[0].count(0)
            co = instance.tolist()[0].count(1)
            marjotiry = (co > cz) and 1 or 0
            final_predict.append(marjotiry)
            counter = instance.tolist()[0].count(1)
            print('Instance = {0}, Marjority = {1}, Type = {2}, Shape = {3}, Sum = {4}, N one = {5}'.format(i, marjotiry, type(instance), instance.shape, instance.sum(), counter))
            i = i + 1
        return final_predict

In [62]:
model = EnginneringForest(3, name_log='test2.txt')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Size DfSub = (91, 13)
Instance = 0, Marjority = 0, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 65, N one = 65
Instance = 1, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 216, N one = 216
Instance = 2, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 233, N one = 233
Instance = 3, Marjority = 0, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 8, N one = 8
Instance = 4, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 161, N one = 161
Instance = 5, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 253, N one = 253
Instance = 6, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 167, N one = 167
Instance = 7, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 222, N one = 222
Instance = 8, Marjority = 1, Type = <class 'numpy.matrix'>, Shape = (1, 286), Sum = 200, N one = 200
Instance = 9, Marjority = 0, Type = <class 'numpy.matrix'>, Shape = (1, 286

In [57]:
model.matrix_predict[0].tolist()[0].count(1)

[[0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1]]

In [56]:
model.matrix_predict[0]

matrix([[0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
         0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
         0, 1, 1, 1, 0, 0, 1]], dtype=int64)

In [60]:
for instance in model.matrix_predict:
    print("\n", instance.tolist()[0])


 [0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]

 [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1]

 [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]

 [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1